In [1]:
#!pip install pygeohash

In [2]:
import json
from pymongo import MongoClient
from pyspark.sql import SparkSession
import os
import random
from datetime import datetime, timedelta
from pygeohash import encode as geohash
import json
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.3.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0 pyspark-shell'
host_ip = "192.168.1.105" # host ip

In [3]:

spark = (
    SparkSession.builder 
    .master('local[*]') 
    .appName('Streaming HotSpot by Climate') # app name
    .getOrCreate() # get else create
)         

In [4]:
topic = "Lastest_Climate,AQUA_HotSpot,Terra_HotSpot" #topic that listen to 3 topic only


In [5]:
rep_kafka_sdf= (
    spark.readStream
    .format('kafka') #listen from kafka
    .option("failOnDataLoss",False) #if fail, data will not losss
    .option('kafka.bootstrap.servers', f'{host_ip}:9092') #listen to  the specific server
    .option('subscribe', topic) #listen to certain topic
    
    .load()
)

In [6]:
climate_sdf = rep_kafka_sdf.select('value')

In [7]:
client = MongoClient(host_ip, 27017) 
db = client.fit3182
climate_db= db["climate_db"]

In [8]:

  
def process(batch, batch_id): #to process the batch
        incomingData = batch.collect() #collect the current batch
        climate_report,hotspot_report={},{}
       
        json_fdata=[ json.loads(report.value)   for report in incomingData]
        
        curdate =None
        #PARTITION based on station
        for report in json_fdata: # for each json object
            latitude = float(report['latitude'])
            longitude = float(report['longitude'])
            gh_val = geohash(latitude, longitude,precision=3) # hash into a dict with precision 3 based on latitude and longitude
            if int(report['publisher'])==1:
                report['hotspot']=[]
                climate_report[gh_val]=report #if it is climate report
            else:
                # print("publisher:",report['publisher']) 
                 if gh_val not in hotspot_report: # if it is hotsport report
                        hotspot_report[gh_val]=[report]
                 else:
                    
                     hotspot_report[gh_val]+=[report]
                        
         #find which gh key has existed in climate report
       
        for climgh in climate_report.keys(): #based on the existed climate report 
          
            curdate =climate_report[climgh]['date'] # to get the current date
            
            #print(curdate)
            if climgh in  hotspot_report:
                    gh_llmerge={} #get the same geohash baesd on geohash precison 5 and partitioned in dictionary
                    for hotsp in hotspot_report[climgh]: # merge by geohash of precision 5
                        gh_val = geohash(latitude, longitude,precision=5)
                        if gh_val not in gh_llmerge:    
                            gh_llmerge[gh_val]=[hotsp]
                        else:
                            gh_llmerge[gh_val]+=[hotsp]
                    for hotspots in gh_llmerge.values(): # for each partition partition again by interval of 10 mintues
                        avg_time_merge=[[[]for i in range(6)]for i in range(24) ] 
                        for hp in hotspots:
                          
                            
                            hours, minutes, _ =hp['published_time'].split(':')   # find the times of mionutes and hours
                            hours = int(hours) # to know which to partition
                            minutes = int(minutes)//10 # to know which to partition
                            avg_time_merge[hours][minutes].append(hp) # append to its
                    merged_hotspot=[]
                    for i in range(24):
                        for j in range(6):
                            if len(avg_time_merge[i][j])>1: # if the partition has more than 1 element
                                conf_sum, surftemp_sum, lat_sum, long_sum, count = 0, 0, 0, 0, len(avg_time_merge[i][j]) # attrivutes to be averaged
                                for hp in avg_time_merge[i][j]: # sum it up
                                    conf_sum += float(hp['confidence'])
                                    surftemp_sum += float(hp['surface_temperature_celcius'])
                                    lat_sum += float(hp['latitude'])
                                    long_sum += float(hp['longitude'])
                                    
                                    
                                #average it
                                avg_conf, avg_surftemp, avg_lat, avg_long =round( conf_sum / count,3), round(surftemp_sum / count,3), round(lat_sum / count,4), round(long_sum / count,4)
                                avg_time_merge[i][j][0]['confidence'] = str(avg_conf)
                                avg_time_merge[i][j][0]['surface_temperature_celcius'] =str(avg_surftemp)
                                avg_time_merge[i][j][0]['latitude'] = str(avg_lat)
                                avg_time_merge[i][j][0]['longitude'] =str(avg_long)
                            
                            if len(avg_time_merge[i][j])>0: # if there is an or more elemet in the partition
                                print("GHI_w/m2:",float(climate_report[climgh]["GHI_w/m2"]))
                                print("air_temperature_celcius:",float(climate_report[climgh]["air_temperature_celcius"]))
                                #check what is the cause
                                print( float(climate_report[climgh]["air_temperature_celcius"]) > 20 and float(climate_report[climgh]["GHI_w/m2"]) >180)
                                if  float(climate_report[climgh]["air_temperature_celcius"]) > 20 and float(climate_report[climgh]["GHI_w/m2"]) >180:
                                    avg_time_merge[i][j][0]["cause"]="natural"
                                else:
                                    avg_time_merge[i][j][0]["cause"]="other"
                                
                                curdate_object = datetime.strptime(curdate,'%d/%m/%Y') # get the current date

                                formatted_date = curdate_object.strftime('%Y-%m-%d')
                                print(formatted_date)
                                avg_time_merge[i][j][0]['publisher']=(random.randrange(2,3))
                                #print(avg_time_merge[i][j][0]['published_time'])
                                # to give hotspot a datetime
                                avg_time_merge[i][j][0]['datetime'] =  formatted_date + " " +avg_time_merge[i][j][0]['published_time']
                                merged_hotspot.append(avg_time_merge[i][j][0]) # merged hotspot for the climate report
                    climate_report[climgh]['hotspot']=merged_hotspot  # embedded into the climate report based on criteria
            climate_db.insert_one(climate_report[climgh]) # insert the climate reprot to climate database
        print("Current Date :", curdate )
        #prove if it is inserted or not
        results = climate_db.find({"date": curdate}) # to find the database
        for document in results:
                print(document)
         

In [9]:
writer = (
    climate_sdf.writeStream.format("Console").trigger(processingTime='10 seconds') #process in a day
    .option("checkpointLocation", "./lastest_clim_report_sdf_checkpoints") # save checkpoint
    .outputMode('append').foreachBatch(process) #process each batch 
)

In [10]:
try:
    query = writer.start()
    query.awaitTermination()
except KeyboardInterrupt:
    print('Interrupted by CTRL-C. Stopping query.')
finally:
    query.stop()

GHI_w/m2: 119.0
air_temperature_celcius: 14.0
False
2024-05-03
Current Date : 04/05/2024
{'_id': ObjectId('665060cacc3c552cc44e79c8'), 'latitude': '-35.957', 'longitude': '141.088', 'air_temperature_celcius': '9', 'relative_humidity': '42.4', 'windspeed_knots': '7.2', 'max_wind_speed': '9.9', 'precipitation ': ' 0.00I', 'GHI_w/m2': '82', 'publisher': 1, 'station': '987401', 'date': '04/05/2024', 'hotspot': [{'latitude': '-36.2889', 'longitude': '141.7692', 'confidence': '64', 'surface_temperature_celcius': '51', 'publisher': 3, 'published_time': '09:20:49', 'cause': 'other', 'datetime': '2024-05-04 09:20:49'}]}
{'_id': ObjectId('66509b6e4ef9c94177e9e56e'), 'latitude': '-35.937', 'longitude': '145.607', 'air_temperature_celcius': '10', 'relative_humidity': '43.3', 'windspeed_knots': '2.6', 'max_wind_speed': '7', 'precipitation ': ' 0.00I', 'GHI_w/m2': '91', 'publisher': 1, 'station': '987401', 'date': '04/05/2024', 'hotspot': []}
GHI_w/m2: 154.0
air_temperature_celcius: 18.0
False
2024-

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/conda/lib/python3.8/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/conda/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


Interrupted by CTRL-C. Stopping query.
